In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import cv2
from typing import Tuple, List
import os
import sys
from tqdm import tqdm

In [2]:
print(f'Tensorflow version: {tf.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'OpenCV version: {cv2.__version__}')
!python --version

Tensorflow version: 2.3.1
Pandas version: 1.1.1
NumPy version: 1.18.5
OpenCV version: 4.4.0


Python 3.6.10 :: Anaconda, Inc.


In [3]:
"""
We turn this on to prevent tensorflow from throwing a fit about
things that take longer than the batch training in the model.fit
call (namely, the tensorboard callback can take more time to
execute than the batch training iteration itself).

NOTE: This can be disabled simply by commenting it out. If you
think there is a weird tensorflow issue happening, do that to see
the full tensorflow logs during runtime.
"""
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
modelName = "mDenseNet"

## Utility Functions

In [5]:
def getDirsAndClasses(root: str, file: str) -> Tuple[List[str], List[int]]:
    imageDirs = []
    classes = []
    line = ""
    with open(root + file, "r") as f:
        for line in tqdm(f):
            imageDir, clazz = line.split()
            imageDirs.append(imageDir)
            classes.append(int(clazz))
    return imageDirs, classes

In [6]:
def preprocessImage(img: np.ndarray) -> tf.Tensor:
    return img.astype(np.float16) / 255.0

In [7]:
def makeOneHot(value: int, size: int) -> np.ndarray:
    onehot = np.zeros(size)
    onehot[value] = 1
    return onehot

In [8]:
def balanceData(data: pd.DataFrame) -> pd.DataFrame:
    data = data.groupby("class")
    data = data.apply(lambda x: x.sample(data.size().min()).reset_index(drop=True))
    return data.reset_index(drop=True)

In [9]:
def makeDenseBlock(groupCount: int, inputs):
    blockConcats = []
    x = layers.BatchNormalization()(inputs)
    x = layers.Conv2D(filters=16, kernel_size=(1, 1), activation="relu", padding="same")(x)
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", padding="same")(x)
    blockConcats.append(x)
    for count in range(groupCount):
        x = layers.Concatenate()(blockConcats) if len(blockConcats) > 1 else x
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters=16, kernel_size=(1, 1), activation="relu", padding="same")(x)
        x = layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", padding="same")(x)
        blockConcats.append(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=16, kernel_size=(1, 1), activation="relu", padding="same")(x)
    x = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    return x

In [10]:
def makeModel(inputShape: Tuple[int], modelName=modelName) -> keras.Model:
    inputs = keras.Input(shape=inputShape, name="Input")
    x = layers.Conv2D(filters=16, kernel_size=(7, 7), strides=(5, 5), activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    
    x = makeDenseBlock(groupCount=2, inputs=x)
    
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(units=2, activation="softmax")(x)

    return keras.Model(inputs=inputs, outputs=outputs, name=modelName)

## Global Variables

In [11]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="accuracy",
        min_delta=0.01,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=f'logs/{modelName}',
        write_graph=True,
        write_images=True
    )
]

root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
# train_imageDirs, train_classes = getDirsAndClasses(root, "\\LABELS\\train.txt")
# test_imageDirs, test_classes = getDirsAndClasses(root, "\\LABELS\\test.txt")
# val_imageDirs, val_classes = getDirsAndClasses(root, "\\LABELS\\val.txt")
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\all.txt")

classDict = {
    0: "free",
    1: "busy"
}

batchSize = 128

144965it [00:00, 891326.49it/s]


## Data Acquisition and Preprocessing

In [12]:
data = pd.DataFrame([
            {
                #"image": preprocessImage(cv2.imread(root + "\\PATCHES\\" + filename)),
                "image": root + "\\PATCHES\\" + filename,
                "class": clazz,
                "weather": filename[0]
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
        ])

144965it [00:00, 1237953.96it/s]


In [13]:
data.groupby("class")["class"].value_counts()

class  class
0      0        65658
1      1        79307
Name: class, dtype: int64

In [14]:
data.groupby("weather")["weather"].value_counts()

weather  weather
O        O          44243
R        R          37544
S        S          63178
Name: weather, dtype: int64

In [15]:
data.groupby(["class", "weather"])["class"].value_counts()

class  weather  class
0      O        0        21067
       R        0        18926
       S        0        25665
1      O        1        23176
       R        1        18618
       S        1        37513
Name: class, dtype: int64

In [16]:
classes = list(data.groupby("class").groups.keys())

In [17]:
data["onehot"] = data["class"].apply(
    func=lambda x: makeOneHot(classes.index(x), len(classes))
)

### Split the data into train and test subsets

In [18]:
train = data.groupby("class").sample(frac=0.8)
train.groupby("class")["class"].value_counts()

class  class
0      0        52526
1      1        63446
Name: class, dtype: int64

In [19]:
test = data.drop(train.index).reset_index(drop=True)
test.groupby("class")["class"].value_counts()

class  class
0      0        13132
1      1        15861
Name: class, dtype: int64

In [20]:
train = train.reset_index(drop=True)

In [21]:
train = tf.data.Dataset.from_tensor_slices(
    (
        np.array([preprocessImage(cv2.imread(image)) for image in tqdm(train["image"].values.tolist())]),
        np.array(train["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(train),
    reshuffle_each_iteration=True
).batch(batchSize)

100%|█████████████████████████████████| 115972/115972 [02:40<00:00, 721.43it/s]


In [22]:
test = tf.data.Dataset.from_tensor_slices(
    (
        np.array([preprocessImage(cv2.imread(image)) for image in tqdm(test["image"].values.tolist())]),
        np.array(test["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(test),
    reshuffle_each_iteration=True
).batch(batchSize)

100%|███████████████████████████████████| 28993/28993 [00:45<00:00, 643.65it/s]


## Model Definition

In [23]:
model = makeModel(inputShape=(150, 150, 3))

In [24]:
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [25]:
model.summary()

Model: "mDenseNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 29, 29, 16)   2368        Input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 14, 14, 16)   64          max_pooling2d[0][0]              
__________________________________________________________________________________________

## Training and Evaluation

In [26]:
model.fit(
    train,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
907/907 [==============================] - 68s 75ms/step - loss: 0.3540 - accuracy: 0.9632
Epoch 2/100
907/907 [==============================] - 86s 95ms/step - loss: 0.3370 - accuracy: 0.9758
Epoch 3/100
907/907 [==============================] - 100s 110ms/step - loss: 0.3333 - accuracy: 0.9793
Epoch 4/100
907/907 [==============================] - 99s 109ms/step - loss: 0.3298 - accuracy: 0.9832
Epoch 5/100
907/907 [==============================] - 96s 106ms/step - loss: 0.3294 - accuracy: 0.9833
Epoch 00005: early stopping


In [27]:
loss, accuracy = model.evaluate(test)

227/227 [==============================] - 11s 49ms/step - loss: 0.3384 - accuracy: 0.9743


In [28]:
model.predict(test)

array([[7.1847424e-12, 1.0000000e+00],
       [4.2664658e-09, 1.0000000e+00],
       [1.0000000e+00, 3.4779415e-08],
       ...,
       [3.8443974e-11, 1.0000000e+00],
       [4.8857949e-12, 1.0000000e+00],
       [9.9999952e-01, 4.2752657e-07]], dtype=float32)

## Results

In [29]:
print(f'Test Loss: {loss}\nTest Accuracy: {accuracy}')

Test Loss: 0.33843106031417847
Test Accuracy: 0.9743041396141052


In [30]:
model.save(f'Models/{modelName}')